In [ ]:
#Preliminary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import re
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


### **AADHAR ENROLMENT DATASET**

This dataset provides aggregated information on **Aadhaar enrolments** across various demographic and geographic levels. It includes variables such as the date of enrollment, state, district, PIN code, and age-wise categories (0–5 years, 5–17 years, and 18 years and above). The dataset captures both temporal and spatial patterns of enrolment activity, enabling detailed descriptive, comparative, and trend analysis.

In [ ]:
enrol_df1 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_enrolment_0_500000.csv")
enrol_df2 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_enrolment_500000_1000000.csv")
enrol_df3 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_enrolment_1000000_1006029.csv")

#Merging datasets one below another
enrol_df = pd.concat([enrol_df1, enrol_df2, enrol_df3], ignore_index=True)
print(f"\nShape of enrol_df1:{enrol_df1.shape}")
print(f"\nShape of enrol_df2:{enrol_df2.shape}")
print(f"\nShape of enrol_df3:{enrol_df3.shape}")
print(f"\nShape of enrol_df:{enrol_df.shape}")

In [ ]:
enrol_df.head()

In [ ]:
x = enrol_df[enrol_df['state'] == '100000']
print(f"{x['age_0_5'].sum()}\n")
print(f"{x['age_5_17'].sum()}\n")
print(f"{x['age_18_greater'].sum()}\n")

In [ ]:
enrol_df_groupby_district = enrol_df.groupby(['state','district']).agg(Age_0_5=('age_0_5','sum'), Age_5_17=('age_5_17','sum'), Age_18_plus=('age_18_greater','sum')).reset_index()
enrol_df_groupby_district.head(20)

In [ ]:
enrol_df_groupby_state = enrol_df.groupby('state').agg(Age_0_5=('age_0_5','sum'), Age_5_17=('age_5_17','sum'), Age_18_plus=('age_18_greater','sum')).reset_index()
enrol_df_groupby_state['total'] = enrol_df_groupby_state['Age_0_5'] + enrol_df_groupby_state['Age_5_17'] + enrol_df_groupby_state['Age_18_plus']
enrol_df_groupby_state.sort_values(by='total', ascending=False, inplace=True)
enrol_df_groupby_state

### **Aadhaar Demographic Update dataset**

This dataset captures aggregated information related to **updates made to residents’ demographic data linked to Aadhaar**, such as name, address, date of birth, gender, and mobile number. It provides insights into the frequency and distribution of demographic changes across different time periods and geographic levels (state, district, and PIN code).

In [ ]:
demographic_df1 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_demographic_0_500000.csv")
demographic_df2 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_demographic_500000_1000000.csv")
demographic_df3 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_demographic_1000000_1500000.csv")
demographic_df4 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_demographic_1500000_2000000.csv")
demographic_df5 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_demographic_2000000_2071700.csv")

#Merging datasets one below another
demographic_df = pd.concat([demographic_df1, demographic_df2, demographic_df3, demographic_df4, demographic_df5], ignore_index=True)
print(f"\nShape of demographic_df1:{demographic_df1.shape}")
print(f"\nShape of demographic_df2:{demographic_df2.shape}")
print(f"\nShape of demographic_df3:{demographic_df3.shape}")
print(f"\nShape of demographic_df4:{demographic_df4.shape}")
print(f"\nShape of demographic_df5:{demographic_df5.shape}")
print(f"\nShape of demographic_df:{demographic_df.shape}")

In [ ]:
demographic_df.head()

### **Aadhaar Biometric Update dataset**

This dataset contains aggregated information on biometric updates (modalities such as fingerprints, iris, and face). It reflects the periodic revalidation or correction of biometric details, especially for children transitioning into adulthood.

In [ ]:
biometric_df1 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_biometric_0_500000.csv")
biometric_df2 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_biometric_500000_1000000.csv")
biometric_df3 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_biometric_1000000_1500000.csv")
biometric_df4 = pd.read_csv(r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\api_data_aadhar_biometric_1500000_1861108.csv")

#Merging datasets one below another
biometric_df = pd.concat([biometric_df1, biometric_df2, biometric_df3, biometric_df4], ignore_index=True)
print(f"\nShape of biometric_df1:{biometric_df1.shape}")
print(f"\nShape of biometric_df2:{biometric_df2.shape}")
print(f"\nShape of biometric_df3:{biometric_df3.shape}")
print(f"\nShape of biometric_df4:{biometric_df4.shape}")
print(f"\nShape of biometric_df:{biometric_df.shape}")

In [ ]:
biometric_df.head()

### **Parsing Date**

In [ ]:
def _parse_date(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])
    df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()  # month start
    return df

bio  = _parse_date(biometric_df)
demo = _parse_date(demographic_df)
enr  = _parse_date(enrol_df)

### **Age column to numeric**

In [ ]:
#age column to numeric
def _age_cols(df: pd.DataFrame):
    base = {"date", "month", "state", "district", "pincode"}
    cols = [c for c in df.columns if c not in base]
    # keep only numeric-like columns
    num_cols = []
    for c in cols:
        if pd.api.types.is_numeric_dtype(df[c]) or df[c].astype(str).str.match(r"^-?\d+(\.\d+)?$").mean() > 0.8:
            num_cols.append(c)
    # force numeric
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
    return num_cols

BIO_AGE  = _age_cols(bio)
DEMO_AGE = _age_cols(demo)
ENR_AGE  = _age_cols(enr)


### **Monthly Aggregate at Pin/District**

In [ ]:
def monthly_aggregate(df: pd.DataFrame, level: str, value_cols: list[str]) -> pd.DataFrame:
    """
    level: "pin" or "district"
    returns monthly sums by level.
    """
    if level == "pin":
        keys = ["month", "state", "district", "pincode"] #group by month + state + district + pincode
    elif level == "district":
        keys = ["month", "state", "district"]            #group by month + state + district
    else:
        raise ValueError("level must be 'pin' or 'district'")

    out = (
        df.groupby(keys, as_index=False)[value_cols]
          .sum()
          .sort_values(keys)
          .reset_index(drop=True)
    )
    # total across age columns (handy for spikes/forecast)
    out["total"] = out[value_cols].sum(axis=1)
    return out

bio_pin_m   = monthly_aggregate(bio,  "pin",      BIO_AGE)
bio_dist_m  = monthly_aggregate(bio,  "district", BIO_AGE)
demo_pin_m  = monthly_aggregate(demo, "pin",      DEMO_AGE)
demo_dist_m = monthly_aggregate(demo, "district", DEMO_AGE)
enr_pin_m   = monthly_aggregate(enr,  "pin",      ENR_AGE)
enr_dist_m  = monthly_aggregate(enr,  "district", ENR_AGE)

# ----------------------------

### **System wide Aadhar activity by Age Group**

In [ ]:
enr_pin_m.head()

In [ ]:
df_enr = enr_pin_m.copy()
age_cols = ENR_AGE

monthly = df_enr.groupby("month", as_index=False)[age_cols].sum()

plt.figure(figsize=(11, 5))

plt.stackplot(
    monthly["month"],
    monthly["age_0_5"],
    monthly["age_5_17"],
    labels=["Age 0–5", "Age 5–17"],
    colors=["#FFA07A", "#20B2AA"]
)

plt.xlabel("Month")
plt.ylabel("Total Activity")
plt.title("January spike indicates system-wide surge; post-Jan enrolment volumes stabilize")
plt.xticks(rotation=45)
plt.legend(loc="upper right")

# --- annotation (add AFTER plotting, BEFORE tight_layout/show)
total_by_month = monthly[["age_0_5", "age_5_17"]].sum(axis=1)

# If you specifically want to mark the first month (as in your snippet)
plt.annotate(
    "System-wide spike",
    xy=(monthly["month"].iloc[0], total_by_month.iloc[0]),
    xytext=(monthly["month"].iloc[1], total_by_month.max() * 0.9),
    arrowprops=dict(arrowstyle="->")
)

plt.tight_layout()
plt.show()


### **Spike detection/Severity Scoring**

The severity score is constructed as a multiplicative composite of three interpretable components: 

* relative magnitude (absolute month-over-month percent change), 
* statistical unusualness (a robust z-score of the month-to-month change using median and MAD), and 
* impact/scale (a log-transformed current-month volume). 

This design reflects a standard “risk = likelihood × impact” philosophy: the robust z-score captures how unlikely the change is relative to the location’s own historical behavior (reducing sensitivity to outliers and non-normality), the percent change captures how large the shift is in relative terms, and the log-volume term ensures the score reflects real-world operational impact without letting very large regions dominate purely by size.

Severity score can be as a prioritization and response tool. The score helps identify which events demand immediate attention because they combine large change, rarity, and high operational impact. High-severity spikes can trigger actions such as surge staffing, temporary capacity expansion, targeted audits, or advance planning for expected drives. Aggregated over time and locations, severity scores also reveal districts that experience intense stress during spike months, informing where contingency plans, buffers, and preventive interventions will deliver the greatest benefit.

In [ ]:
def add_spike_metrics(monthly_df: pd.DataFrame, group_keys: list[str], value_col: str = "total") -> pd.DataFrame:
    """
    Adds MoM absolute/percent changes + robust z-score on change + severity score.
    severity = (abs(pct_change) capped) * robust_z_change * log1p(current_value)
    """
    df = monthly_df.copy()
    df = df.sort_values(group_keys + ["month"])

    # MoM changes per group
    #Create a groupby object (one group = one PIN time series
    #g represents multiple separate time series:
    #one for each unique (state, district, pincode) combination.
    g = df.groupby(group_keys, sort=False)

    #For each group (PIN let's say)
    #-moves the values down by 1 row
    #-so current row can reference its previous month’s value
    df["prev"] = g[value_col].shift(1)                #last month’s total
    df["abs_change"] = df[value_col] - df["prev"]     #current_total − prev_total
    #If a PIN has its first month in the data: prev becomes NaN because there’s no earlier row.

    # percent change (safe)
    df["pct_change"] = np.where(df["prev"].fillna(0) == 0, np.nan, df["abs_change"] / df["prev"])

    # robust z-score of abs_change within each group (using MAD)
    #It takes a series of changes within one group (PIN) and scores how unusual each change is compared to that PIN’s own history.
    #What it is computing -- A) med: median of the series  B) mad: median absolute deviation from the median
    #robust z = (x − median) / (1.4826 * MAD)
    def _robust_z(s: pd.Series) -> pd.Series:
        med = np.nanmedian(s)
        mad = np.nanmedian(np.abs(s - med))
        if mad == 0 or np.isnan(mad):
            return pd.Series(np.zeros(len(s)), index=s.index, dtype=float)
        return (s - med) / (1.4826 * mad) #Evaluating the z_score while 1.4826 * MAD = Standard Deviation for normal distribution

    df["z_change"] = g["abs_change"].transform(_robust_z).abs() #Passing absolute change series through _robust_z
    #z_change ~ 0 --> Normal; z_change >=3 --> Significant anomaly


    # cap pct_change to avoid infinite blow-ups from tiny denominators
    pct = df["pct_change"].abs().clip(lower=0, upper=20)  # bounded percent-change magnitude
    
    
    #Severity score ranks regions by how unusually large and operationally impactful their recent activity changes are. 
   
    df["severity"] = (pct.fillna(0) + 0.1) * (df["z_change"].fillna(0) + 0.1) * np.log1p(df[value_col])
    #pct --> How big was the month-to-month change in percentage?
    #z_change --> How unusual was this change for this PIN historically or how surprising is this change for this area?
    #log(1+x) --> How big is the real-world impact of this anomaly without letting large regions overpower the analysis?
    # If either pct or z_change is 0, the product would become 0 and you’d lose information.+ 0.1 makes severity still non-zero for mild anomalies and prevents hard zeros.

    # flag spikes (TRUE if)
    #Condition 1: Top 1% by severity globally --> severity >= 99th percentile, OR
    #Condition 2: z_change >= 3 (very unusual for that PIN) AND pct >= 1 (≥ 100% change)
    
    df["is_spike"] = (df["severity"] >= df["severity"].quantile(0.99)) | ((df["z_change"] >= 3) & (pct >= 1))
    return df

In [ ]:
# Apply on PIN-level (best for anomaly surfacing)
bio_spikes_pin  = add_spike_metrics(bio_pin_m,  group_keys=["state","district","pincode"], value_col="total") #Aadhar Biometric spikes at PIN level
demo_spikes_pin = add_spike_metrics(demo_pin_m, group_keys=["state","district","pincode"], value_col="total") #Aadhar Demographic spikes at PIN level
enr_spikes_pin  = add_spike_metrics(enr_pin_m,  group_keys=["state","district","pincode"], value_col="total") #Aadhar Enrollment spikes at PIN level

In [ ]:
%%time
# Apply on District-level (best for anomaly surfacing)
bio_spikes_district  = add_spike_metrics(bio_dist_m,  group_keys=["state","district"], value_col="total") #Aadhar Biometric spikes at district level
demo_spikes_district = add_spike_metrics(demo_dist_m, group_keys=["state","district"], value_col="total") #Aadhar Demographic spikes at district level
enr_spikes_district  = add_spike_metrics(enr_dist_m,  group_keys=["state","district"], value_col="total") #Aadhar Enrollment spikes at district level

In [ ]:
#Severity in descending order (Aadhar enrollment spikes at PIN level)
enr_spikes_pin[enr_spikes_pin["is_spike"] == True].sort_values("severity", ascending=False).head()

In [ ]:
#Severity in descending order (Aadhar enrollment spikes at District level)
enr_spikes_district[enr_spikes_district["is_spike"] == True].sort_values("severity", ascending=False).head()

### **Top anomalies by month**

Anomalies typically represent unusual surges, drops, or disruptions that cannot be explained by normal variability. In this analysis, anomaly detection is applied at a granular month-level to measure how different a given month’s activity is compared to that location’s own historical behavior, rather than against a global average.

Methodologically, the approach combines month-to-month change, relative percentage change, and a robust z-score based on the median absolute deviation (MAD), which is well established in the literature as a stable alternative to standard deviation when data contain outliers or non-normal distributions. This allows the model to detect true operational shocks while remaining resistant to noise and scale effects across districts of different sizes. Each detected anomaly is further quantified using a severity score, which integrates the magnitude of change, its statistical rarity, and the underlying activity volume, ensuring that both unusualness and real-world impact are captured.

The utility of anomaly detection lies in its ability to produce actionable, time-specific signals—highlighting when and where abnormal events occur, such as special enrolment drives, system outages, or backlog clearances. These anomaly signals form the evidentiary layer of the analysis and are subsequently aggregated to inform higher-level risk metrics, such as spike frequency and severity in the Instability Score. In this way, anomaly detection serves as the event-level diagnostic foundation, complementing district-level instability analysis and enabling both immediate operational response and longer-term planning.

In [ ]:
def top_anomalies_by_month(spike_df: pd.DataFrame, topn: int = 20) -> pd.DataFrame:
    """
    Returns top N spike anomalies for each month ranked by severity (descending).
    Works for both PIN-level spikes (has pincode) and district-level spikes (no pincode).
    Only includes rows where is_spike == True.
    """
    df = spike_df.copy()

    # keep valid months and only spikes
    df = df[df["month"].notna()].copy()
    df = df[df["is_spike"] == True].copy()

    if df.empty:
        # return empty dataframe with whatever columns would have existed
        return df

    # rank within month by severity (highest = rank 1)
    df["rank_in_month"] = df.groupby("month")["severity"].rank(method="first", ascending=False)

    out = (
        df[df["rank_in_month"] <= topn]
        .sort_values(["month", "rank_in_month"], ascending=[True, True])
        .reset_index(drop=True)
    )

    # Choose identifier columns based on availability
    id_cols = ["month", "state", "district"]
    if "pincode" in out.columns:
        id_cols.append("pincode")

    # Common metric columns (keep only those that exist)
    metric_cols = [
        "total", "prev", "abs_change", "pct_change", "z_change",
        "severity", "is_spike", "rank_in_month"
    ]
    keep_cols = [c for c in (id_cols + metric_cols) if c in out.columns]

    return out[keep_cols]


In [ ]:
%%time
#Anomaly detection at Pincode level
bio_top_anoms_by_pin_month  = top_anomalies_by_month(bio_spikes_pin,  topn=20) #topn can change
demo_top_anoms_by_pin_month = top_anomalies_by_month(demo_spikes_pin, topn=20)
enr_top_anoms_by_pin_month  = top_anomalies_by_month(enr_spikes_pin,  topn=20)

In [ ]:
%%time
#Anomaly detection at District level
bio_top_anoms_by_dist_month  = top_anomalies_by_month(bio_spikes_district,  topn=20)
demo_top_anoms_by_dist_month = top_anomalies_by_month(demo_spikes_district, topn=20)
enr_top_anoms_by_dist_month  = top_anomalies_by_month(enr_spikes_district,  topn=20)

In [ ]:
enr_top_anoms_by_dist_month.head()

In [ ]:
enr_top_anoms_by_pin_month.head()

### **Instability Score**

Instability Score is a composite indicator that summarizes how operationally unpredictable a district’s Aadhaar activity is over time. Rather than relying on raw monthly volumes, it integrates four complementary signals: 

* volatility (how much monthly activity fluctuates relative to its average), 
* spike frequency (how often abnormal surges occur), 
* peak stress (how extreme the highest-load month is compared to normal), and 
* spike intensity (how severe those abnormal surges are when they occur). 

Together, these dimensions capture both chronic instability and event-driven risk, an approach widely used in operations and risk management literature where composite indices are favored for their robustness and interpretability over single metrics.

For a decision maker, the Instability Score functions as a prioritization and planning tool. A higher score indicates districts or pins that are harder to manage operationally and more prone to service disruptions without proactive intervention.

The score translates complex time-series behavior into a single, actionable measure that supports informed resource allocation, risk mitigation, and policy decisions.

In [ ]:
"""
    level="district" -> one row per (state, district)
    level="pin"      -> one row per (state, district, pincode)

    Returns one row per (state, district/pin) with:
      - cov (weight = 0.30): {Std/Mean}, 
      - spike_freq (weight = 0.25): {is_spike = TRUE / no of months}, 
      - peak_factor (weight = 0.20): {Max Total / Mean}
      - severity metric (weight = 0.25): aggregated severity of spikes in that district
      - normalized components
      - final instability_score (0-100)

    Inputs:
      monthly_df: output of monthly_aggregate(..., level="district") OR similar
      spike_df: output of add_spike_metrics() at district or pin level

     severity_agg:
      - sum: total severity across spiky PINs in a district-month (captures distributed stress)
      - max: worst spike in that district-month (captures extreme local pocket)
      - mean: average severity across spiky PINs (captures typical spike intensity), how intense is a typical spike pocket?
      
    severity_stat = mean: 
        What it emphasizes
        - Chronic, everyday stress
        - Repeated moderate problems
        - Long-term operational load
        What it ignores
        - Rare but very dangerous months
        - Extreme one-off stress events
    severity_stat = p95:
        What it emphasizes
        - Risk of extreme stress events
        - Preparedness for worst-case scenarios
        - Planning for rare but high-impact spikes
        What it ignores
        - Everyday operational load
        - Chronic moderate problems
        - Normal months
    """


import numpy as np
import pandas as pd

def _robust_minmax(s: pd.Series, low_q=0.05, high_q=0.95) -> pd.Series:
    s = s.astype(float)
    lo = s.quantile(low_q)
    hi = s.quantile(high_q)
    if pd.isna(lo) or pd.isna(hi) or hi == lo:
        return pd.Series(np.zeros(len(s)), index=s.index, dtype=float)
    return (s.clip(lo, hi) - lo) / (hi - lo)

def instability_score_with_severity(
    monthly_df: pd.DataFrame,
    spike_df: pd.DataFrame,
    level: str = "district",            # "district" or "pin"
    value_col: str = "total",
    min_months: int = 4,
    weights: dict | None = None,
    severity_agg: str = "sum",          # "sum" or "max" or "mean"
    severity_stat: str = "p95",         # "mean" or "p95"
    peak_mode: str = "ratio"            # "ratio" or "log_ratio"
) -> pd.DataFrame:
   
    # ---- Defaults
    if weights is None:
        # Must sum to 1.0 for clean 0–100 interpretation
        weights = {"cov": 0.30, "spike_freq": 0.25, "peak": 0.20, "severity": 0.25}

    # ---- Validate mode params
    if level not in {"district", "pin"}:
        raise ValueError("level must be 'district' or 'pin'")
    if severity_agg not in {"sum", "max", "mean"}:
        raise ValueError("severity_agg must be one of: 'sum', 'max', 'mean'")
    if severity_stat not in {"mean", "p95"}:
        raise ValueError("severity_stat must be one of: 'mean', 'p95'")
    if peak_mode not in {"ratio", "log_ratio"}:
        raise ValueError("peak_mode must be 'ratio' or 'log_ratio'")

    # ---- Group keys
    keys = ["state", "district"] if level == "district" else ["state", "district", "pincode"]

    # ---- Validate required columns
    required_monthly = set(keys + ["month", value_col])
    missing_monthly = required_monthly - set(monthly_df.columns)
    if missing_monthly:
        raise ValueError(f"monthly_df missing required columns: {missing_monthly}")

    required_spike = set(keys + ["month", "is_spike", "severity"])
    missing_spike = required_spike - set(spike_df.columns)
    if missing_spike:
        raise ValueError(f"spike_df missing required columns: {missing_spike}")

    # ---- Sort monthly data
    df = monthly_df.copy().sort_values(keys + ["month"])

    # 1) Base metrics per group
    base = (
        df.groupby(keys, as_index=False)
          .agg(
              months=("month", "nunique"),
              mean=(value_col, "mean"),
              std=(value_col, "std"),
              max_total=(value_col, "max"),
          )
    )

    base["cov"] = np.where(base["mean"] == 0, np.nan, base["std"] / base["mean"])
    base["peak_factor"] = np.where(base["mean"] == 0, np.nan, base["max_total"] / base["mean"])
    if peak_mode == "log_ratio":
        base["peak_factor"] = np.log1p(base["peak_factor"])

    # Ensure enough history
    base = base[base["months"] >= min_months].copy()

    # 2) Spike frequency (align spikes to months present in monthly_df)
    spikes_only = spike_df[spike_df["is_spike"] == True].copy()

    # Keep only months that exist in monthly_df for the same group (prevents time-range mismatch)
    valid_months = df[keys + ["month"]].drop_duplicates()
    spikes_only = spikes_only.merge(valid_months, on=keys + ["month"], how="inner")

    spike_months = (
        spikes_only.drop_duplicates(subset=keys + ["month"])
        .groupby(keys, as_index=False)
        .agg(spike_months=("month", "nunique"))
    )

    out = base.merge(spike_months, on=keys, how="left")
    out["spike_months"] = out["spike_months"].fillna(0).astype(int)
    out["spike_freq"] = np.where(out["months"] == 0, 0.0, out["spike_months"] / out["months"])

    # 3) Severity metric
    # Aggregate severity at group-month level
    if severity_agg == "sum":
        group_month_sev = spikes_only.groupby(keys + ["month"], as_index=False)["severity"].sum()
    elif severity_agg == "max":
        group_month_sev = spikes_only.groupby(keys + ["month"], as_index=False)["severity"].max()
    else:  # mean
        group_month_sev = spikes_only.groupby(keys + ["month"], as_index=False)["severity"].mean()

    group_month_sev = group_month_sev.rename(columns={"severity": "group_month_severity"})

    # Summarize across months per group
    if severity_stat == "mean":
        sev = (
            group_month_sev.groupby(keys, as_index=False)["group_month_severity"]
            .mean()
            .rename(columns={"group_month_severity": "severity_metric"})
        )
    else:  # p95
        sev = (
            group_month_sev.groupby(keys, as_index=False)["group_month_severity"]
            .quantile(0.95)
            .rename(columns={"group_month_severity": "severity_metric"})
        )

    out = out.merge(sev, on=keys, how="left")
    out["severity_metric"] = out["severity_metric"].fillna(0.0)

    # 4) Normalize components
    out["cov_norm"] = _robust_minmax(out["cov"].fillna(0))
    out["spike_norm"] = _robust_minmax(out["spike_freq"].fillna(0))
    out["peak_norm"] = _robust_minmax(out["peak_factor"].fillna(0))
    out["severity_norm"] = _robust_minmax(out["severity_metric"].fillna(0))

    # 5) Final score (0–100)
    out["instability_score"] = 100.0 * (
        weights["cov"] * out["cov_norm"] +
        weights["spike_freq"] * out["spike_norm"] +
        weights["peak"] * out["peak_norm"] +
        weights["severity"] * out["severity_norm"]
    )

    out = out.sort_values("instability_score", ascending=False).reset_index(drop=True)

    cols = (
        keys + [
            "months", "mean", "std", "cov",
            "max_total", "peak_factor",
            "spike_months", "spike_freq",
            "severity_metric",
            "cov_norm", "spike_norm", "peak_norm", "severity_norm",
            "instability_score"
        ]
    )
    return out[cols]

    

In [ ]:
#Instability score at district level
bio_dist_inst = instability_score_with_severity(
    monthly_df=bio_dist_m,
    spike_df=bio_spikes_district,
    level="district",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)

demo_dist_inst = instability_score_with_severity(
    monthly_df=demo_dist_m,
    spike_df=demo_spikes_district,
    level="district",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)

enr_dist_inst = instability_score_with_severity(
    monthly_df=enr_dist_m,
    spike_df=enr_spikes_district,
    level="district",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)


In [ ]:
#Instability for Enrollment dataset at district level
enr_dist_inst.head()

In [ ]:
enr_dist_inst["instability_score"].value_counts(bins=10).sort_index()

In [ ]:
#Instability Scores for PINs
bio_pin_inst = instability_score_with_severity(
    monthly_df=bio_pin_m,
    spike_df=bio_spikes_pin,
    level="pin",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)

demo_pin_inst = instability_score_with_severity(
    monthly_df=demo_pin_m,
    spike_df=demo_spikes_pin,
    level="pin",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)

enr_pin_inst = instability_score_with_severity(
    monthly_df=enr_pin_m,
    spike_df=enr_spikes_pin,
    level="pin",
    severity_agg="sum",
    severity_stat="p95",
    peak_mode="ratio"
)


In [ ]:
enr_pin_inst["instability_score"].value_counts(bins=10).sort_index()

In [ ]:
#Instability for Enrollment dataset at Pincode level
enr_pin_inst.head(10)

### **Top X contribution in instability**

Top-X contribution analysis is a concentration metric used to quantify how much of a system’s total instability is driven by a small subset of units (e.g., districts or PIN codes). Rather than only identifying the most unstable entities, it measures the cumulative share of instability accounted for by the top X% of units, providing insight into the distributional structure of risk across the system.

The primary objective of Top-X contribution analysis is to answer the question:

*“Is instability widespread across the system, or is it concentrated in a small number of hotspots?”*

By expressing instability in cumulative terms (e.g., “Top 10% of districts contribute 55% of total instability”), this metric enables:

* Prioritization of interventions, by identifying whether targeting a small subset of districts can significantly reduce system-wide risk.
* Efficient resource allocation, avoiding uniform responses when problems are structurally concentrated.
* Strategic planning, by distinguishing between systemic instability and localized operational stress.

This makes Top-X contribution especially valuable for decision makers who must choose where limited monitoring, staffing, or corrective capacity should be deployed.

In [ ]:
import numpy as np
import pandas as pd

def top_x_contribution(df: pd.DataFrame, value_col: str = "instability_score", x: float = 0.10) -> dict:
    if not (0 < x <= 1):
        raise ValueError("x must be between 0 and 1")

    s = pd.to_numeric(df[value_col], errors="coerce").fillna(0.0).clip(lower=0)

    if s.sum() == 0:
        return {"top_share_units_pct": 0.0, "contribution_share_pct": 0.0, "n_top_units": 0}

    sorted_vals = s.sort_values(ascending=False)
    n = len(sorted_vals)
    n_top = max(1, int(np.ceil(x * n)))

    top_sum = sorted_vals.iloc[:n_top].sum()
    total_sum = sorted_vals.sum()

    return {
        "top_share_units_pct": round(100 * n_top / n, 2),
        "contribution_share_pct": round(100 * top_sum / total_sum, 2),
        "n_top_units": n_top
    }

def gini_coefficient(values: pd.Series) -> float:
    x = pd.to_numeric(values, errors="coerce").fillna(0.0).to_numpy()
    x = np.clip(x, 0, None)
    if x.sum() == 0:
        return 0.0
    x = np.sort(x)
    n = len(x)
    cum = np.cumsum(x)
    g = (n + 1 - 2 * (cum.sum() / cum[-1])) / n
    return float(g)

def _id_cols_for_level(df: pd.DataFrame, level: str) -> list[str]:
    if level == "district":
        return ["district", "state"]
    if level == "pin":
        # if state/district available, include them for context
        cols = []
        if "pincode" in df.columns:
            cols.append("pincode")
        if "district" in df.columns:
            cols.append("district")
        if "state" in df.columns:
            cols.append("state")
        return cols if cols else ["pincode"]
    raise ValueError("level must be 'district' or 'pin'")

def format_top_units(df: pd.DataFrame, level: str, topn: int = 5, value_col: str = "instability_score") -> str:
    top = df.sort_values(value_col, ascending=False).head(topn)
    id_cols = _id_cols_for_level(df, level)

    parts = []
    for _, r in top.iterrows():
        label = ", ".join([f"{c}={r[c]}" for c in id_cols if c in top.columns])
        parts.append(f"{label} → {r[value_col]:.1f}")
    return "; ".join(parts)

def reason_mix(df: pd.DataFrame, col: str = "reason_code") -> str | None:
    if col not in df.columns:
        return None
    vc = df[col].value_counts(normalize=True).head(5) * 100
    return ", ".join([f"{k}:{v:.0f}%" for k, v in vc.items()])

def generate_dataset_insight(
    df_inst: pd.DataFrame,
    dataset_name: str,
    level: str = "district",            # "district" or "pin"
    x_list: list[float] = [0.05, 0.10, 0.20],
    topn: int = 5,
    value_col: str = "instability_score"
) -> str:
    if value_col not in df_inst.columns:
        raise ValueError(f"{dataset_name}: '{value_col}' column not found")

    # Check id columns exist
    if level == "district":
        required = {"state", "district"}
        missing = required - set(df_inst.columns)
        if missing:
            raise ValueError(f"{dataset_name}: missing required district columns: {missing}")
    elif level == "pin":
        if "pincode" not in df_inst.columns:
            raise ValueError(f"{dataset_name}: missing required pin column: 'pincode'")
    else:
        raise ValueError("level must be 'district' or 'pin'")

    g = gini_coefficient(df_inst[value_col])

    if g >= 0.50:
        conc_word = "highly concentrated"
    elif g >= 0.35:
        conc_word = "moderately concentrated"
    else:
        conc_word = "fairly distributed"

    conc_lines = []
    for x in x_list:
        res = top_x_contribution(df_inst, value_col=value_col, x=x)
        conc_lines.append(f"top {int(x*100)}% contribute {res['contribution_share_pct']}%")

    unit_word = "districts" if level == "district" else "PIN codes"
    conc_text = "; ".join(conc_lines)

    top_text = format_top_units(df_inst, level=level, topn=topn, value_col=value_col)

    mix = reason_mix(df_inst, col="reason_code")
    mix_text = f" Reason mix: {mix}." if mix else ""

    return (
        f"{dataset_name} ({unit_word}): Instability is {conc_word} (Gini={g:.2f}). "
        f"Concentration: {conc_text} of total instability. "
        f"Highest-risk {unit_word}: {top_text}.{mix_text}"
    )


In [ ]:
print(generate_dataset_insight(enr_dist_inst, "Biometric Updates", level="district"))


In [ ]:
print(generate_dataset_insight(enr_pin_inst, "Biometric Updates", level="pin"))

### **K Means**

We use K-means clustering here to discover natural groups of districts/PINs that behave similarly, rather than evaluating each location in isolation.

**What K-means adds beyond ranking or scoring?**

* Instability score ranks severity, but it does not explain why a district is unstable.
* K-means clusters explain structure, grouping districts with similar instability drivers even if their total scores differ.

Two districts/pin-codes may have the same instability score:

* one driven by frequent small spikes,
* another driven by a single extreme peak.

K-means separates them into different clusters, enabling tailored interventions.

In [ ]:
import numpy as np
import pandas as pd


def kmeans_cluster_districts(
    df: pd.DataFrame,
    feature_cols: list[str],
    k: int | None = None,
    k_min: int = 3,
    k_max: int = 8,
    random_state: int = 42
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Adds a 'cluster' column using KMeans on feature_cols.
    If k=None, chooses best k by silhouette.
    Returns: (df_with_cluster, cluster_profile)
    """
    out = df.copy()
    X = out[feature_cols].fillna(0.0).to_numpy()

    # Standardize for KMeans fairness
    Xs = StandardScaler().fit_transform(X)

    if k is None:
        best_k, best_score = None, -1
        for kk in range(k_min, k_max + 1):
            model = KMeans(n_clusters=kk, n_init=30, random_state=random_state)
            labels = model.fit_predict(Xs)
            if len(np.unique(labels)) < 2:
                continue
            score = silhouette_score(Xs, labels)
            if score > best_score:
                best_score, best_k = score, kk
        k = best_k if best_k is not None else 4

    kmeans = KMeans(n_clusters=k, n_init=50, random_state=random_state)
    out["cluster"] = kmeans.fit_predict(Xs)

    # Cluster profile for interpretation
    profile = (
        out.groupby("cluster", as_index=False)
           .agg(
               n=("cluster", "size"),
               **{f"avg_{c}": (c, "mean") for c in feature_cols},
               avg_instability=("instability_score", "mean") if "instability_score" in out.columns else ("cluster","size")
           )
           .sort_values("avg_instability", ascending=False)
           .reset_index(drop=True)
    )

    return out, profile

def plot_clusters_pca(
    df: pd.DataFrame,
    feature_cols: list[str],
    title: str = "District Clusters (PCA projection)",
    hover_cols: list[str] | None = None
):
    if hover_cols is None:
        hover_cols = ["state", "district", "instability_score", "reason_code"]

    plot_df = df.dropna(subset=feature_cols + ["cluster"]).copy()

    # keep only hover columns that exist
    hover_cols = [c for c in hover_cols if c in plot_df.columns]

    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    import plotly.express as px

    X = plot_df[feature_cols].fillna(0.0).to_numpy()
    Xs = StandardScaler().fit_transform(X)

    pca = PCA(n_components=2, random_state=42)
    coords = pca.fit_transform(Xs)
    plot_df["pca_1"] = coords[:, 0]
    plot_df["pca_2"] = coords[:, 1]

    fig = px.scatter(
        plot_df,
        x="pca_1",
        y="pca_2",
        color="cluster",
        hover_data=hover_cols,
        title=title
    )
    fig.update_layout(height=650)
    fig.show()
    return fig


In [ ]:
def plot_cluster_profile(profile_df: pd.DataFrame, feature_cols: list[str], title="Cluster Driver Profile"):
    # profile has avg_feature columns (avg_cov_norm etc.)
    cols = ["cluster", "n"] + [f"avg_{c}" for c in feature_cols]
    d = profile_df[cols].copy()

    long = d.melt(id_vars=["cluster","n"], var_name="metric", value_name="value")
    long["metric"] = long["metric"].str.replace("avg_", "", regex=False)

    fig = px.bar(long, x="metric", y="value", color="cluster", barmode="group",
                 title=title, hover_data=["n"])
    fig.update_layout(height=520)
    fig.show()
    return fig

**Plotting Enrollment District Instability**

In [ ]:
features = ["cov_norm", "spike_norm", "peak_norm", "severity_norm"]  # drop severity_norm if not using it

enr_inst_district_k, enr_inst_district_cluster_profile = kmeans_cluster_districts(enr_dist_inst, feature_cols=features, k=None)
#print(enr_inst_district_cluster_profile)

plot_clusters_pca(
    enr_inst_district_k,
    feature_cols=features,
    title="Enrollment Instability — District Clusters",
    hover_cols=["state", "district", "instability_score", "cluster"]
)


In [ ]:
plot_cluster_profile(enr_inst_district_cluster_profile, features, title="Enrollment District Instability — Cluster Profiles")


### **Decoding cluster profile -- Aadhar Enrollment - District wise - Instability**

🔵 **Cluster 0 (Blue) — Stable / Low-Risk**

*Profile: cov_norm≈0.075, spike_norm=0, peak_norm≈0.063, severity_norm=0*

This cluster represents districts with highly stable enrollment behavior. Low normalized volatility and peak stress indicate that demand remains close to its typical level, while the absence of spikes and severity suggests no abnormal surges over the observed months. In operational terms, these districts behave predictably and require only routine monitoring and standard staffing. Cluster 0 can serve as a benchmark baseline for expected system behavior.

Operational implications:

* Maintain standard staffing and processes
* Use as a benchmark baseline for expected performance
* No intervention required beyond routine monitoring

Probable Reason Code: STABLE

🟣 **Cluster 1 (Purple) — Extreme Mixed Instability / High-Risk Hotspots**

*Profile: cov_norm≈0.913, spike_norm≈0.644, peak_norm≈0.918, severity_norm≈0.874*

This is the highest-risk cluster, characterized by simultaneous elevation across all four instability dimensions. High CoV indicates chronic month-to-month turbulence; high spike frequency shows that abnormal months occur repeatedly; high peak stress implies extreme capacity-shock months; and high severity confirms that spikes are operationally intense, not merely statistical noise. In practice, these are the districts most likely to trigger service congestion or SLA risk and should be prioritized for comprehensive intervention—buffer capacity, proactive forecasting, special monitoring, and root-cause investigation.

Operational implications:

* Highest priority for intervention
* Deploy buffer capacity and surge staffing
* Enable proactive monitoring and early-warning systems
* Conduct root-cause analysis (infrastructure, demand shocks, policy drives)

Probable Reason Code: MIXED

🌸 **Cluster 2 (Pink) — Moderate Mixed Variability / Managed Instability**

*Profile: cov_norm≈0.498, spike_norm≈0.556, peak_norm≈0.522, severity_norm≈0.136*

Cluster 2 reflects districts with moderate instability driven by regular fluctuations and recurring spikes, but with low spike intensity. The combination of mid-level CoV, spike frequency, and peak factor suggests these areas experience operational variability (likely due to routine administrative cycles or predictable drives), yet the low severity indicates that spike events are generally contained and manageable. Operationally, this cluster benefits from improved scheduling and early-warning triggers rather than major capacity expansion.

Operational implications:

* Improve scheduling discipline and demand smoothing
* Use rolling forecasts and adaptive staffing
* Monitor trends rather than respond to incidents

Probable Reason Code: VOLATILE

🟠 **Cluster 3 (Orange) — Peaky-Volatile Without Spikes / Structural Swings**

*Profile: cov_norm≈0.639, spike_norm=0, peak_norm≈0.656, severity_norm=0*

This cluster shows structural variation and peak pressure without being classified as “spiky.” Higher volatility and peak factor indicate enrollment demand swings and elevated maximum months relative to baseline, but the absence of spike frequency/severity implies these changes are either gradual, seasonal, or do not meet the anomaly threshold used for spike classification. Operationally, this cluster represents districts where capacity planning should focus on seasonality management and forecasting, rather than incident response.

Operational implications:

* Plan seasonal or policy-driven capacity surges
* Temporary staffing during known peak windows
* Focus on capacity planning, not anomaly response

Probable Reason Code: PEAKY

🟡 **Cluster 4 (Yellow) — Spiky-Low-Volume / Event-Driven Surges**

*Profile: cov_norm≈0.072, spike_norm≈0.619, peak_norm≈0.064, severity_norm≈0.150*

Cluster 4 is defined by frequent spikes occurring on an otherwise stable baseline. Low CoV and peak factor suggest typical volumes are steady, but spike_norm is high—meaning spikes occur repeatedly. Severity is low-to-moderate, implying these spikes are not extremely intense but are persistent. This pattern is consistent with event-driven behavior (scheduled drives, periodic backlogs, outreach campaigns) that causes recurring surges without fundamentally destabilizing the baseline. Operationally, this cluster is best managed through event calendars, pre-emptive staffing for known spike windows, and queue management, rather than permanent capacity increase.

Operational implications:

* Plan seasonal or policy-driven capacity surges
* Temporary staffing during known peak windows
* Focus on capacity planning, not anomaly response

Probable Reason Code: SPIKY

In [ ]:
features = ["cov_norm", "spike_norm", "peak_norm", "severity_norm"]  # drop severity_norm if not using it

enr_inst_pin_k, enr_pin_cluster_profile = kmeans_cluster_districts(enr_pin_inst, feature_cols=features, k=None)
#print(enr_pin_cluster_profile)

plot_clusters_pca(
    enr_inst_pin_k,
    feature_cols=features,
    title="Enrollment Instability — Pincode Clusters",
    hover_cols=["state", "district", "pincode", "instability_score", "cluster"]
)

In [ ]:
plot_cluster_profile(enr_pin_cluster_profile, features, title="Enrollment Instability — Cluster Profiles (Pincode)")

### **Decoding cluster profile -- Aadhar Enrollment - Pincode wise - Instability**

🔵 **Cluster 0 (Blue) — Extreme Mixed Instability Hotspots**

*Profile: cov_norm ≈ 0.96, spike_norm ≈ 0.81, peak_norm ≈ 0.96, severity_norm ≈ 0.91*

This cluster represents high-risk pincode hotspots where all dimensions of instability are simultaneously elevated. Enrollment activity in these pincodes is highly volatile month-to-month, experiences frequent spikes, reaches extreme peak loads, and those spikes are operationally severe. This pattern indicates localized breakdowns or intense operational stress, often driven by concentrated population demand, repeated special drives, or infrastructure constraints. These pincodes should be the top priority for intervention, requiring granular monitoring, surge capacity, and possibly structural fixes at the enrolment-center level.

Operational implication: Requires deep root-cause analysis and multi-pronged intervention.

Probable Reason Code: MIXED

🌸 **Cluster 1 (Pink) — Stable Low-Activity Pincodes**

*Profile: cov_norm ≈ 0.11, spike_norm = 0, peak_norm ≈ 0.09, severity_norm = 0*

Cluster 1 captures pincodes with stable and predictable enrollment behavior. Low volatility and peak stress indicate consistent demand, and the complete absence of spikes and severity suggests no abnormal surges across the observation window. These pincodes form the operational baseline of the system. They require minimal intervention beyond routine operations and can be used as reference benchmarks when evaluating abnormal behavior elsewhere.

Operational implication: Routine operations; no special action required.

Probable Reason Code: STABLE


🟠 **Cluster 2 (Orange) — Spiky Event-Driven Pincodes**

*Profile: cov_norm ≈ 0.19, spike_norm ≈ 0.86, peak_norm ≈ 0.17, severity_norm ≈ 0.41*

This cluster is characterized by frequent spike occurrences on an otherwise stable baseline. Volatility and peak stress remain low, but spike frequency is high, and severity is moderate—indicating repeated, localized surges that are noticeable but not catastrophic. These patterns are consistent with event-driven behavior, such as enrollment camps, periodic backlog clearances, or targeted outreach programs. Operationally, these pincodes benefit most from anticipatory planning (event calendars, temporary staffing) rather than permanent capacity expansion.

Operational implication: Routine operations; no special action required.

Probable Reason Code: SPIKY

🟡 **Cluster 3 (Yellow) — Peaky but Non-Spiky Structural Load**

*Profile: cov_norm ≈ 0.85, spike_norm = 0, peak_norm ≈ 0.84, severity_norm = 0*

Cluster 3 represents pincodes with large structural swings and high peak months, but without being flagged as anomalous spikes. High volatility and peak factor suggest strong seasonal or cyclical demand patterns, yet the absence of spike frequency and severity implies these changes are gradual or expected, rather than sudden disruptions. These pincodes require forecast-driven capacity planning and seasonal staffing adjustments, not anomaly response mechanisms.

Operational implication: Event-driven planning, early warnings, and short-term staffing buffers.

Probable Reason Code: PEAKY

### **Reason Code**

* **VOLATILE**
 
    -Chronic month-to-month instability

    -CoV high relative to other districts

    -Interpretation: The district’s workload is structurally unstable (planning + staffing mismatch, inconsistent throughput, or irregular operations).

* **SPIKY**

    -Repeated abnormal events (drives, outages, backlog clearances).

    -spike months occur often across the district timeline

    -Interpretation: The district is usually stable, but repeatedly gets disrupted by episodic events.

* **PEAKY**

    -One or two extreme months causing capacity shock.

    -A single “mountain” month. Flat before and after. That one month explains most of the perceived instability.

    -Capacity planning problem, not chronic instability. You should plan temporary infrastructure / staff surge for known seasonal or campaign-like peak windows.

* **MIXED**

    -Multiple instability drivers → needs deeper investigation.

    -Both structural instability and event shocks exist; needs deeper investigation and multi-pronged intervention.

We decompose district instability into chronic variability (CoV), anomaly frequency (spike months), capacity shocks (peak factor), and spike intensity (severity). We assign a reason code only when one normalized driver clearly dominates; otherwise we label the district as MIXED to avoid false certainty and trigger deeper diagnostic review.

In [ ]:
def add_reason_code_with_severity(
    df: pd.DataFrame,
    margin: float = 0.10,
    cov_col: str = "cov_norm",
    spike_col: str = "spike_norm",
    peak_col: str = "peak_norm",
    severity_col: str = "severity_norm",
    out_col: str = "reason_code",
) -> pd.DataFrame:
    """
    Assigns a dashboard-friendly reason code when Instability Score has 4 components:
      - VOLATILE  : cov_norm dominates
      - SPIKY     : spike_norm dominates (spike frequency)
      - PEAKY     : peak_norm dominates (peak factor)
      - INTENSE   : severity_norm dominates (spike intensity)
      - MIXED     : no clear dominant driver (top-2 within `margin`)

    Parameters
    ----------
    margin : float
        If (top - second) <= margin, mark as MIXED.
        Example: margin=0.10 means top driver must be >= 0.10 higher than second to "dominate".
    """
    out = df.copy()

    required = [cov_col, spike_col, peak_col, severity_col]
    missing = [c for c in required if c not in out.columns]
    if missing:
        raise ValueError(f"Missing required normalized columns: {missing}")

    comps = out[required].to_numpy()

    # Top and second-top values per row (district)
    sorted_vals = np.sort(comps, axis=1)   # ascending
    top = sorted_vals[:, -1]
    second = sorted_vals[:, -2]

    # Which component is maximum: 0=cov, 1=spike, 2=peak, 3=severity
    argmax = np.argmax(comps, axis=1)

    # MIXED if top two are close
    is_mixed = (top - second) <= margin

    labels = np.array(["VOLATILE", "SPIKY", "PEAKY", "INTENSE"], dtype=object)

    out[out_col] = np.where(is_mixed, "MIXED", labels[argmax])

    # Optional: dominant component name for debugging/explainability
    dom_names = np.array(["cov", "spike_freq", "peak_factor", "severity"], dtype=object)
    out["dominant_component"] = np.where(is_mixed, "mixed", dom_names[argmax])

    return out


In [ ]:
#Reason codes for instability at district level
bio_instability_district_reason1  = add_reason_code_with_severity(bio_dist_inst)
demo_instability_district_reason1 = add_reason_code_with_severity(demo_dist_inst)
enr_instability_district_reason1  = add_reason_code_with_severity(enr_dist_inst)

In [ ]:
#Reason codes for instability at Pincode level
bio_instability_pin_reason1  = add_reason_code_with_severity(bio_pin_inst)
demo_instability_pin_reason1 = add_reason_code_with_severity(demo_pin_inst)
enr_instability_pin_reason1  = add_reason_code_with_severity(enr_pin_inst)

**Reason Code -- District wise**

In [ ]:
enr_instability_district_reason1.head(10)[["state", "district", "instability_score", "reason_code","dominant_component"]]

In [ ]:
enr_instability_district_reason1["reason_code"].value_counts()

**Reason Code -- Pincode wise**

In [ ]:
enr_instability_pin_reason1.head(10)[["state", "district", "pincode", "instability_score", "reason_code","dominant_component"]]

In [ ]:
enr_instability_pin_reason1["reason_code"].value_counts()

### **Forecast next month load**

Exponential Smoothing

In [ ]:
def forecast_next_month_ewma(
    monthly_df: pd.DataFrame,
    group_keys: list[str],
    value_col: str = "total",
    alpha: float = 0.5
) -> pd.DataFrame:
    """
    One-step-ahead forecast using Simple Exponential Smoothing (EWMA).

    Forecast(t+1) = alpha * y(t) + (1 - alpha) * Forecast(t)
    Prediction interval is heuristic, based on EWMA std.
    """

    # Defensive copy & proper ordering
    df = monthly_df.copy().sort_values(group_keys + ["month"])
    g = df.groupby(group_keys, sort=False)

    # EWMA level (forecast signal)
    df["ewma"] = g[value_col].transform(
        lambda s: s.ewm(alpha=alpha, adjust=False).mean()
    )

    # EWMA-based volatility (uncertainty signal)
    df["ewm_std"] = g[value_col].transform(
        lambda s: s.ewm(alpha=alpha, adjust=False).std(bias=False)
    )

    # Take last observed month per group
    last = df.groupby(group_keys, as_index=False).tail(1).copy()

    # Forecast month = next calendar month
    last["forecast_month"] = (
        last["month"] + pd.offsets.MonthBegin(1)
    ).dt.to_period("M").dt.to_timestamp()

    # One-step-ahead forecast
    last["forecast"] = last["ewma"]

    # Heuristic 95% prediction interval
    last["pi_low"] = (last["forecast"] - 1.96 * last["ewm_std"].fillna(0)).clip(lower=0)
    last["pi_high"] = (last["forecast"] + 1.96 * last["ewm_std"].fillna(0)).clip(lower=0)

    cols = (
        group_keys
        + ["forecast_month", "forecast", "pi_low", "pi_high", "month", value_col]
    )

    return last[cols].rename(
        columns={"month": "last_month", value_col: "last_value"}
    )

In [ ]:
bio_forecast_district_next_es  = forecast_next_month_ewma(bio_dist_m,  group_keys=["state","district"], value_col="total")
demo_forecast_district_next_es = forecast_next_month_ewma(demo_dist_m, group_keys=["state","district"], value_col="total")
enr_forecast_district_next_es  = forecast_next_month_ewma(enr_dist_m,  group_keys=["state","district"], value_col="total")

In [ ]:
bio_forecast_pin_next_es  = forecast_next_month_ewma(bio_pin_m,  group_keys=["state","district","pincode"], value_col="total")
demo_forecast_pin_next_es = forecast_next_month_ewma(demo_pin_m, group_keys=["state","district","pincode"], value_col="total")
enr_forecast_pin_next_es  = forecast_next_month_ewma(enr_pin_m,  group_keys=["state","district","pincode"], value_col="total")

### **Top “consistent performers”**

In [ ]:
def consistent_performers(monthly_df: pd.DataFrame, level: str = "district",
                          value_col: str = "total", min_months: int = 4,
                          volume_quantile: float = 0.6, topn: int = 50) -> pd.DataFrame:
    """
    Finds stable performers:
    - low CoV and low median MoM % change
    - but also above a volume threshold to avoid tiny-activity regions
    """
    if level == "district":
        keys = ["state","district"]
    elif level == "pin":
        keys = ["state","district","pincode"]
    else:
        raise ValueError("level must be 'district' or 'pin'")

    df = monthly_df.copy().sort_values(keys + ["month"])
    df["prev"] = df.groupby(keys)[value_col].shift(1)
    df["mom_abs_pct"] = np.where(df["prev"].fillna(0) == 0, np.nan, (df[value_col]-df["prev"]).abs()/df["prev"])

    agg = (
        df.groupby(keys, as_index=False)
          .agg(
              months=("month","nunique"),
              total_sum=(value_col,"sum"),
              mean=(value_col,"mean"),
              std=(value_col,"std"),
              median_mom_abs_pct=("mom_abs_pct","median"),
              max_total=(value_col,"max"),
          )
    )
    agg["cov"] = np.where(agg["mean"] == 0, np.nan, agg["std"]/agg["mean"])

    agg = agg[agg["months"] >= min_months].copy()
    vol_thresh = agg["total_sum"].quantile(volume_quantile)
    agg = agg[agg["total_sum"] >= vol_thresh].copy()

    # stability score: lower is better
    agg["stability_score"] = (agg["cov"].fillna(0) * 0.7) + (agg["median_mom_abs_pct"].fillna(0) * 0.3)
    agg = agg.sort_values(["stability_score","total_sum"], ascending=[True, False]).head(topn).reset_index(drop=True)
    return agg

In [ ]:
bio_consistent_districts  = consistent_performers(bio_dist_m,  level="district", value_col="total", min_months=4, volume_quantile=0.6, topn=50)
demo_consistent_districts = consistent_performers(demo_dist_m, level="district", value_col="total", min_months=4, volume_quantile=0.6, topn=50)
enr_consistent_districts  = consistent_performers(enr_dist_m,  level="district", value_col="total", min_months=4, volume_quantile=0.6, topn=50)

In [ ]:
enr_consistent_districts.head(10)

In [ ]:
bio_consistent_pin  = consistent_performers(bio_pin_m,  level="pin", value_col="total", min_months=4, volume_quantile=0.6, topn=50)
demo_consistent_pin = consistent_performers(demo_pin_m, level="pin", value_col="total", min_months=4, volume_quantile=0.6, topn=50)
enr_consistent_pin  = consistent_performers(enr_pin_m,  level="pin", value_col="total", min_months=4, volume_quantile=0.6, topn=50)

In [ ]:
enr_consistent_pin.head(10)

### **Save outputs**

In [ ]:
OUT_DIR = r"C:\Users\Abhay\OneDrive\Desktop\Hackathons\UIDAI\Output"
#Aadhar Enrollment spikes
enr_spikes_pin.to_csv(f"{OUT_DIR}/enr_spikes_pin.csv", index=False)
enr_spikes_district.to_csv(f"{OUT_DIR}/enr_spikes_district.csv", index=False)

#Aadhar Enrollment Anomalies
enr_top_anoms_by_dist_month.to_csv(f"{OUT_DIR}/enr_top_anoms_by_dist_month.csv", index=False)
enr_top_anoms_by_pin_month.to_csv(f"{OUT_DIR}/enr_top_anoms_by_pin_month.csv", index=False)

#Aadhar Enrollment instability
enr_pin_inst.to_csv(f"{OUT_DIR}/enr_instability_pin.csv", index=False)
enr_dist_inst.to_csv(f"{OUT_DIR}/enr_instability_district.csv", index=False)

#Aadhar Enrollment reason codes
enr_instability_district_reason1.to_csv(f"{OUT_DIR}/enr_instability_district_reason.csv", index=False)
enr_instability_pin_reason1.to_csv(f"{OUT_DIR}/enr_instability_pin_reason.csv", index=False)

#Aadhar Enrollment Forecast
enr_forecast_district_next_es.to_csv(f"{OUT_DIR}/enr_forecast_district_next_es.csv", index=False)
enr_forecast_pin_next_es.to_csv(f"{OUT_DIR}/enr_forecast_pin_next_es.csv", index=False)

#Aadhar Enrollment consistent performers
enr_consistent_districts.to_csv(f"{OUT_DIR}/enr_consistent_districts.csv", index = False)
enr_consistent_pin.to_csv(f"{OUT_DIR}/enr_consistent_pin.csv", index = False)

print("Done. Key outputs created:")
print(f"CSVs saved under: {OUT_DIR}/")